  ### Dataset curation: making the zika-colombia fasta files for main build and supplemental analyses
  
This notebook contains code needed to go from a raw download of all Zika genomes in `nextstrain/fauna` to the input fasta file for the zika-colombia specific analyses (which are custom `nextstrain/augur` builds). 

In [1]:
#import libraries
from Bio import SeqIO
import pandas as pd
import numpy as np
import random
import os

## First thing that I am going to do is curate the dataset for the primary analysis. Curations steps that I would like to perform include:

1. Removing sequences from any geographic areas that I do not want included in the analysis (e.g. Singapore)

2. Removing any sequences that were up on GenBank, but were not actually published on, and for which we didn't receive the author's permission to include in our analysis.

3. Removing repeat sequences of the same strains that were sequenced at different cell culture passage counts.

4. Removing any sequences in which there are many sites with N's (i.e. very little of the genome sequence has informative base calls)

In [2]:
#write any functions that I want to have for this work.

## replace n's with gaps, and count n's in sequences (alignment checks)
def count_n(sequence):
    "counts numbers of N's in a sequence to perform QC and see how many non-informative sites exist"
    counter = 0
    for base in sequence:
        if base == 'n':
            counter +=1
    return counter

def sample_fasta_dict_without_replacement(dictionary,n_samples_to_draw):
    """randomly samples a Seq IO dictionary without replacement."""
    samples = random.sample(dictionary.items(), k=n_samples_to_draw)
    return samples

## 1. Removing sequences from geographic areas outside the Americas and Oceania

In [5]:
# Paths to files, keeping relational so that paths should work if someone downloads the repo as is.
fauna_seqs_dict = SeqIO.to_dict(SeqIO.parse('../data/zika-fauna-2018-09-06.fasta', 'fasta'))
print('There are {} sequences downloaded from Fauna.'.format(len(fauna_seqs_dict)))

#Geographic pruning
regions_to_exclude = ['southeast_asia', 'japan_korea', 'china', 'europe', 'africa']
print('Genomes from the following regions will be excluded: {0}, {1}, {2}, and {3}.'.format(regions_to_exclude[0],regions_to_exclude[1],regions_to_exclude[2],regions_to_exclude[3]))

geoPruned_seqs_dict = {fauna_seqs_dict[key].description:fauna_seqs_dict[key].seq for key in fauna_seqs_dict.keys() if key.split('|')[4] not in regions_to_exclude}
print('There are {} sequences meet the geographic criteria.'.format(len(geoPruned_seqs_dict)))


There are 694 sequences downloaded from Fauna.
Genomes from the following regions will be excluded: southeast_asia, japan_korea, china, and europe.
There are 504 sequences meet the geographic criteria.


## 2. Specifying which published and permission-given sequences we can use.

In [48]:
# read in the dataframe that has the permissions information,
# then parse that to select out all strains that can be included in a publishable analysis
# these are the strains that should be used, and form the fauna subset we want.

genome_permissions = pd.read_csv('../data/genome-permissions-2018-09-06.txt', delimiter ='\t')

publishable_strains = []
for i in range(len(genome_permissions)):
    record = genome_permissions.iloc[i]
    if record['permission_to_use'] != 'permission_not_received' and record['preliminarily_include'] == 'yes':
        publishable_strains.append(record['strain_name'])

print("There are {} genomes that we can include in published analyses.".format(len(publishable_strains)))

# using the strains in the publishable_strains list, pull out the full fauna headers (and sequences)
# for each strain that can be published on.
# also making sure we are only dealing with sequences within the countries of interest (Americans and Oceania)
publishable_seqs_dict = {}
for strain in publishable_strains:
    for key in geoPruned_seqs_dict.keys():
        if key.startswith(strain):
            publishable_seqs_dict[key] = geoPruned_seqs_dict[key]        

There are 431 genomes that we can include in published analyses.


## 3. Specifying which Colombian sequences are cell culture passages of the same clinical sample and should NOT be included in the build.

In [56]:
col_flr_strains_to_remove = [strain for strain in publishable_strains if strain.startswith("COL/FLR") and strain != 'COL/FLR/2015']
print("There were {} sequences from different culture passages of COL/FLR/2015 that should be removed.".format(len(col_flr_strains_to_remove)))

publishable_seqs_no_cell_culture_passage_dict = {}
for key in publishable_seqs_dict.keys():
    if key.split('|')[0] not in col_flr_strains_to_remove:
        publishable_seqs_no_cell_culture_passage_dict[key] = publishable_seqs_dict[key]

assert(len(publishable_seqs_no_cell_culture_passage_dict.keys()) == (len(publishable_seqs_dict.keys()) -(len(col_flr_strains_to_remove))))
print("There are now {} sequences that should be included in the build.".format(len(publishable_seqs_no_cell_culture_passage_dict)))


There were 33 sequences from different culture passages of COL/FLR/2015 that should be removed.
There are now 398 sequences that should be included in the build.


## 4. Specifying a quality theshold: Samples must have at least 50% of sites be unambiguous base calls.

In [110]:
n_counts_dict = {}
for key in publishable_seqs_no_cell_culture_passage_dict.keys():
    n_count = count_n(publishable_seqs_no_cell_culture_passage_dict[key])
    n_counts_dict[key] = n_count
            
medium_qual_seqs = {}
for key in n_counts_dict.keys():
    if key.split('|')[5] == 'colombia':
        medium_qual_seqs[key] = publishable_seqs_no_cell_culture_passage_dict[key]
    else:
        if float(n_counts_dict[key])/10769 < 0.5:
            medium_qual_seqs[key] = publishable_seqs_no_cell_culture_passage_dict[key]

print("At this point we have {} sequences that meet quality specifications, are not cell culture passages, and are publishable.".format(len(medium_qual_seqs.keys())))


At this point we have 385 sequences that meet quality specifications, are not cell culture passages, and are publishable.


## 5. Removing sequences that deviate significantly from the molecular clock.

I found these sequences by running iterative Augur builds until no sequences got trimmed from the build for deviation. In this build, sequences whose rate of evolution is more than 4 interquartile distances outside of the clock rate across the dataset are trimmed.

I've added this step here for reproducibility. I want to ensure that the proper input file can be made by running this notebook. So even though I did lots of builds to figure out what sequences should be removed, this list should be complete and result in the correct input file now.

In [118]:
clock_deviant_strains = ['USVI/28/2016', 'mex39/Mexico/2016', 'USA/2016/FL019', 'BRA/2016/FC_DQ12D1', 'DOM/2016/MA_WGS16_013', 
                         'Bahia02', 'Bahia03', 'GTM/2016/Guatemala_2411', 'MEX/2016/mex01', 'MEX/2016/mex37', 'NIC/2016/Nicaragua_3358']

sequences_for_proper_build = {key:value for key,value in medium_qual_seqs.items() if key.split('|')[0] not in clock_deviant_strains}

print("We now have {} sequences that are ready to go into our zika-colombia augur build.".format(len(sequences_for_proper_build)))

We now have 374 sequences that are ready to go into our zika-colombia augur build.


## Cleaning up some strain names...

When I was looking at the input file I saw a couple of strain name errors or things that were confusing that I initially edited by hand. However, in the hopes of making this completely reproducible, I'm putting in some code below to make those edits. 

The issues were that 1) there were a couple of Colombian strains where the sampling date was in 2016, but the strain name said 2015, and 2) the Ecuadorian strain names were really hard to interpret as being from Ecuador, so I wanted to make those strain names more informative.

In [130]:
strain_name_conversion_dict = {'COL/FH19/2015':'COL/FH19/2016',
                              'EcEs062_16':'Ecuador/EcEs062_16/2016',
                              'EcEs089_16':'Ecuador/EcEs089_16/2016',
                              'COL/FH16/2015':'COL/FH16/2016'}

fixed_names_sequences_for_proper_build = {}
for key in sequences_for_proper_build.keys():
    split_name = key.split('|')
    if split_name[0] in strain_name_conversion_dict.keys():
        new_name = strain_name_conversion_dict[split_name[0]] +'|'+ '|'.join(split_name[1:])
        fixed_names_sequences_for_proper_build[new_name] = sequences_for_proper_build[key]
    else:
        fixed_names_sequences_for_proper_build[key] = sequences_for_proper_build[key]

## N masking for site 10375 in the augur alignment

Nanopolish seems to have some issue where at this site specifically it calls the reference rather than a SNP (despite strong evidence in the BAM stack for a SNP, and the fact that all other sequences in the Americas differ from French Polynesian sequences at this site). Since this seems to artifactual, I've N-masked this site in all sequences sequenced on Nanopore (since these are the sequences that are nanopolished).

In [131]:
target_sequence_with_miscall = 'CTTAGTGTTG'
n_masked_sequence_replacement ='CTTANTGTTG'

properly_n_masked_sequences_for_proper_build = {}
for key in fixed_names_sequences_for_proper_build.keys():
    if key.split('|')[4] == 'oceania': #a G at this site is correct for oceania sequences!
        properly_n_masked_sequences_for_proper_build[key] = fixed_names_sequences_for_proper_build[key]
    elif target_sequence_with_miscall in fixed_names_sequences_for_proper_build[key]:
        masked_seq = fixed_names_sequences_for_proper_build[key].replace(target_sequence_with_miscall,n_masked_sequence_replacement)
        properly_n_masked_sequences_for_proper_build[key] = masked_seq
    else:
        properly_n_masked_sequences_for_proper_build[key] = fixed_names_sequences_for_proper_build[key]

## And finally, let's write these sequences out to a fasta file to run in our augur build.

In [132]:
#write sequences to file
with open('../data/input.fasta','w') as file:
    for key in properly_n_masked_sequences_for_proper_build.keys():
        file.write(str('>' + key + '\n' + properly_n_masked_sequences_for_proper_build[key] + '\n'))

# Make input data files for rarefaction curve supplemental analysis

Next up, I want to make a little additional dataset for a supplemental analysis I'm doing, making rarefaction curves to investigate how many introductions one observes given the numbers of sequences sampled. I'm going to do this analysis for sequences from Colombia and sequences from Mexico, and it involves subsampling them down as well. 

In addition to this, I will need to make a "background sequences" file. This fasta will contain all of the sequences from the Americas that are used in the main analysis build EXCEPT for the country for which the subsampling is occurring (e.g. background file for Mexican subsampling analysis will contain all other American sequences in the main build, including all Colombian, but won't have any Mexican seuquences). Later on the in the build subsampled fastas will be concatenated with background sequences fasta in order to make Augur build input files.

In [27]:
all_main_build_sequences = SeqIO.to_dict(SeqIO.parse('../data/publishable-zika-fauna-2018-10-15.fasta', 'fasta'))

background_seqs_no_mexico = {all_main_build_sequences[key].description:all_main_build_sequences[key].seq for key in all_main_build_sequences.keys() if key.split('|')[5] != 'mexico'}

background_seqs_no_colombia = {all_main_build_sequences[key].description:all_main_build_sequences[key].seq for key in all_main_build_sequences.keys() if key.split('|')[5]!= 'colombia'}


In [33]:
with open("../supplemental-analysis/rarefaction-curves/mexico/data/background_seqs_no_mexico.fasta","w") as file:
    for key in background_seqs_no_mexico.keys():
        file.write(str(">" + key + "\n" + background_seqs_no_mexico[key] + "\n" ))
        
with open("../supplemental-analysis/rarefaction-curves/colombia/data/background_seqs_no_colombia.fasta","w") as file:
    for key in background_seqs_no_colombia.keys():
        file.write(str(">"+ key + "\n" + background_seqs_no_colombia[key] + "\n"))

In [5]:
# Making a dictionary that has the Mexican sequences I'd like to use in the rarefaction analysis.
# Needs to be current, publishable, high-quality sequences. Some of these are newer than the original zika-colombia build
# because data lock happened before these sequences were published, so I'm actually pulling Mexican sequences for this analysis
# from a more recent fauna download.

rarefaction_seqs_dict = SeqIO.to_dict(SeqIO.parse('../data/zika.fasta', 'fasta'))
print ('There are {} sequences downloaded from Fauna in the rarefaction import.'.format(len(rarefaction_seqs_dict)))

mexico_seqs_dict = {}
for key in rarefaction_seqs_dict.keys():
    if key.split('|')[5] == 'mexico':
        mexico_seqs_dict[key] = rarefaction_seqs_dict[key]
    
#notably not all of these sequences can be published on, so the following author's sequences should be dropped from the analysis
drop_authors = ['Sevilla-Reyes','Balaraman','Izquierdo', 'Valdespino-Vazquez']
mexican_seqs_for_use = {key:mexico_seqs_dict[key] for key in mexico_seqs_dict.keys() if key.split('|')[10] not in drop_authors}

#I also don't want to include sequences that are 50% N, so checking quality. 
# I'm going to say that the sequences need to be high quality: they need to have at least 80% informative bases.
samples_to_exclude_due_to_quality = []
for key in mexican_seqs_for_use.keys():
    n_count = count_n(mexican_seqs_for_use[key])
    if n_count > (10769*0.2):
        samples_to_exclude_due_to_quality.append(key)
        
high_qual_mexican_seqs_for_use = {key:mexican_seqs_for_use[key] for key in mexican_seqs_for_use.keys() if key not in samples_to_exclude_due_to_quality}
print(len(high_qual_mexican_seqs_for_use))

#Okay, high_qual_mexican_seqs is the set that should be used/subsampled for rarefaction analyses.

There are 745 sequences downloaded from Fauna in the rarefaction import.
51


Now I'm subsampling down both my Colombian and my Mexican sequence dictionaries to make the input datasets for the rarefaction curve analysis, which looks at how many introductions into a country (either Mexico or Colombia) are observed given x numbers of sequences sampled from that country. The idea is to see when this relationship asymptotes, i.e. how many sequences do you need to observe most of the introductions to a country that occurred.

To get the data sets for this analysis, I need to subsample down one countries sequences, and re-run the augur pipelines with the rest of the build the same, and just look at how introductions to that country changed given numbers of sequences from that country obtained. I'll do this both for Colombia and for Mexico, but separately (i.e. any subsampled Mexican sequences will be run with all Colombian sequences and vice versa).

The subsampling scheme is as follows. Try 1 sequence, 2 sequences, 3 sequences ... all the sequences, and look at numbers of introductions observed. For each subsample amount (e.g. 4 sequences) there will be 5 trials. 

In [100]:
random.seed(123456) #setting a seed so that sampling process is reproducible
n_trials_per_subsampling = 5
#okay now I want to randomly sample mexican sequences, where number of trials increments by 1 until the whole dataset is grabbed
#e.g. 
for i in range(1,len(high_qual_mexican_seqs_for_use)): #start at 1 seq, not zero, but doesn't need to do 51 because that would be no sampling.
    print("\nsubsampling ", i, " genomes.\n")
    os.mkdir("../supplemental-analysis/rarefaction-curves/mexico/mexico_{}_seqs".format(i))#make a directory for each subsample number that will hold the 5 trial data.
    for k in range(1,n_trials_per_subsampling+1): #doing this just for readibility, so that trial number is 1 through 5, rather than 0 through 4
        print("working on trial number: ", k)
        subsample = sample_fasta_dict_without_replacement(high_qual_mexican_seqs_for_use,i)
        with open("../supplemental-analysis/rarefaction-curves/mexico/mexico_{}_seqs/mex_{}_seqs_trial_{}.fasta".format(i,i,k),'w') as file:
            for sequence in subsample:
                file.write(str(">"+sequence[0] + '\n' + sequence[1].seq + '\n'))
        
    
#triall=sample_fasta_dict_without_replacement(high_qual_mexican_seqs_for_use,3)
#print(triall[0][1].seq)


subsampling  1  genomes.

working on trial number:  1
working on trial number:  2
working on trial number:  3
working on trial number:  4
working on trial number:  5

subsampling  2  genomes.

working on trial number:  1
working on trial number:  2
working on trial number:  3
working on trial number:  4
working on trial number:  5

subsampling  3  genomes.

working on trial number:  1
working on trial number:  2
working on trial number:  3
working on trial number:  4
working on trial number:  5

subsampling  4  genomes.

working on trial number:  1
working on trial number:  2
working on trial number:  3
working on trial number:  4
working on trial number:  5

subsampling  5  genomes.

working on trial number:  1
working on trial number:  2
working on trial number:  3
working on trial number:  4
working on trial number:  5

subsampling  6  genomes.

working on trial number:  1
working on trial number:  2
working on trial number:  3
working on trial number:  4
working on trial number:  